In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split,cross_val_score,cross_val_predict
import seaborn as sns

In [ ]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

In [ ]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

## merge data for exploration

In [ ]:
test_data['Survived'] = -1
train_data['Set'] = 'Train'
test_data['Set'] = 'Test'

DATA = train_data.append(test_data)
DATA.reset_index(inplace=True)
DATA.dtypes

In [ ]:
women = train_data.loc[train_data.Sex == 'female']['Survived']
rate_women = sum(women)/len(women)

print("% of women who survived:", rate_women)

In [ ]:
men = train_data.loc[train_data.Sex == 'male']['Survived']
rate_men = sum(men)/len(men)

print("% of women who survived:", rate_men)

In [ ]:
train_data.shape

In [ ]:
train_data.info()

In [ ]:
print("nulls?")
train_data.isna().sum()

In [ ]:
print("fill in")
train_data.Embarked.fillna('S', inplace = True)
train_data.isna().sum()

In [ ]:
def print_all_info(data):
    for i in data.select_dtypes(exclude='object').columns:
        print('_________________________________________________________')
        print(i)
        print(data.groupby(['Survived'])[i].describe())
        
print_all_info(train_data)

## feature engineering

### family size

Make it a category

In [ ]:
DATA['FamilySize'] = DATA.SibSp + DATA.Parch + 1
g = sns.catplot(x="FamilySize",y="Survived",data=DATA[DATA.Set == 'Train'], kind="bar", height = 6, palette = "muted")
g = g.set_ylabels("survival probability - FamilySize")

In [ ]:
def familySize_to_cat(size):
    if size == 1: return "alone"
    if size >= 2 and size <= 4: return 'small'
    if size >= 5 and size <= 7: return 'medium'
    if size > 7 : return "large"

DATA['FamilySizeCategory'] = DATA.FamilySize.apply(familySize_to_cat)

g = sns.catplot(x="FamilySizeCategory",y="Survived",data=DATA[DATA.Set == 'Train'], kind="bar", height = 6, palette = "muted")
g = g.set_ylabels("survival probability - FamilySizeCategory")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

sns.set_style('whitegrid')
plt.rc('patch', edgecolor='black')
sns.set_context("notebook", font_scale=1.3, rc={"lines.linewidth": 1.5})

sns.factorplot('Embarked','Survived',data=train_data)

In [ ]:
categorical = ["Embarked"]

DATA = pd.get_dummies(DATA, columns=["Embarked", "FamilySizeCategory"])


print_all_info(DATA)

write_submission = True






## separate data for training

In [ ]:
features = [
    "Pclass", "Sex", "SibSp", "Parch",
    "FamilySizeCategory_alone",
    "FamilySizeCategory_large",
    "FamilySizeCategory_medium",
#     "Embarked_C",
#     "Embarked_S",
#     "Embarked_Q"
]

TRAIN = DATA[DATA.Set == 'Train']
TEST = DATA[DATA.Set == 'Test']


X = pd.get_dummies(TRAIN[features])
y = TRAIN["Survived"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
X.head()

In [ ]:
TEST.head()

In [ ]:
# manual param sweep

reg_params = [1, 10, 100, 1000]

for reg in reg_params:
    model = svm.SVC(probability=True, kernel='rbf', C=reg)
    model.fit(X_train, y_train)
    train_predictions = model.predict(X_test)
    acc = accuracy_score(y_test, train_predictions)
    print("score: ", acc, " reg: ", reg)
    score = cross_val_score(model, X, y, cv=10)

    print("scores: ", score, " reg: ", reg)
    

## automatic hyper param sweep

In [ ]:
# C is the regularization param
tuned_parameters = [
    {"kernel": ["rbf"], "gamma": [1e-3, 1e-4], "C": [1, 10, 100, 1000]},
    {"kernel": ["linear"], "C": [1, 10, 100, 1000]},
]


scores = ["precision", "recall"]

for score in scores:
    
    model = GridSearchCV(svm.SVC(), tuned_parameters, scoring="%s_macro" % score, cv=5)
    model.fit(X_train, y_train)
    
    print("Best parameters set found on development set:")
    print()
    print(model.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = model.cv_results_["mean_test_score"]
    stds = model.cv_results_["std_test_score"]
    for mean, std, params in zip(means, stds, model.cv_results_["params"]):
        print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, model.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

In [ ]:
# code submission

# best model
model = svm.SVC(kernel='rbf', C=1000, gamma=0.001)
model.fit(X_train, y_train)
test_predictions = model.predict(pd.get_dummies(TEST[features]))

output = pd.DataFrame({"PassengerId": TEST.PassengerId, "Survived": test_predictions})
output.to_csv("submission.csv", index=False)


print("Submission saved!")

print(pd.read_csv("submission.csv"))